In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# GPU kontrolü
!nvidia-smi

# Çalışma dizinini sıfırlama
%cd /content

/bin/bash: line 1: nvidia-smi: command not found
/content


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm
from sklearn.cluster import DBSCAN
import torch
from PIL import Image
import pytesseract
from ultralytics import YOLO

# Gerekli kütüphanelerin kurulumu
!pip install -q ultralytics
!pip install -q pytesseract
!apt-get install tesseract-ocr -y
!pip install -q scenedetect[opencv]
!pip install -q supervision

# Scene Detection için gerekli kütüphaneleri içe aktar
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector

# Tesseract OCR'ı yapılandır
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# YOLO modellerini yükle
person_model = YOLO('yolov8x.pt')  # Kişi tespiti için daha büyük model
ocr_model = YOLO('yolov8n.pt')     # Forma numarası için hafif model

def download_models():
    """Gerekli modelleri indir"""
    print("YOLO modellerini indirme...")
    # Modeller zaten import edildiğinde yüklenir

def detect_scenes(video_path, threshold=30.0):
    """
    Videodaki sahne değişimlerini tespit eder
    Args:
        video_path: Video dosyasının yolu
        threshold: Sahne değişimi için eşik değeri
    Returns:
        scenes: Sahne başlangıç ve bitiş frame numaralarının listesi
    """
    print("Sahneleri tespit etme...")

    # VideoManager ve SceneManager oluştur
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    # Video işleme başla
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)

    # Sahneleri al
    scene_list = scene_manager.get_scene_list()

    # Frame numaralarına dönüştür
    scenes = []
    for scene in scene_list:
        scenes.append((scene[0].frame_num, scene[1].frame_num))

    print(f"Toplam {len(scenes)} sahne tespit edildi.")
    return scenes

def detect_and_track_players(video_path, scene, scene_id):
    """
    Sahnedeki oyuncuları tespit eder ve takip eder
    Args:
        video_path: Video dosyasının yolu
        scene: (başlangıç_frame, bitiş_frame) şeklinde sahne bilgisi
        scene_id: Sahne ID'si
    Returns:
        players: Her oyuncu için track ID ve frame'lerini içeren sözlük
    """
    print(f"Sahne {scene_id} için oyuncuları tespit etme ve takip etme...")

    # Video dosyasını aç
    cap = cv2.VideoCapture(video_path)

    # Sahnenin başlangıç frame'ine git
    start_frame, end_frame = scene
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Oyuncular için sözlük
    players = {}
    next_id = 1  # Her sahne için ID'leri sıfırla

    # Oyuncu izleme için önceki tespit sonuçlarını sakla
    prev_boxes = []
    prev_ids = []

    frame_idx = start_frame

    # Sahne boyunca frame'leri işle
    while frame_idx < end_frame:
        ret, frame = cap.read()
        if not ret:
            break

        # Kişileri tespit et
        results = person_model(frame, classes=0)  # Sadece kişileri tespit et (class 0)

        # Tespit edilen kişilerin bounding box'larını al
        current_boxes = []
        for box in results[0].boxes.xyxy.cpu().numpy():
            current_boxes.append(box)

        current_boxes = np.array(current_boxes)

        # İlk frame ise, her kişiye yeni bir ID ata
        if len(prev_boxes) == 0 and len(current_boxes) > 0:
            current_ids = list(range(next_id, next_id + len(current_boxes)))
            next_id += len(current_boxes)
        # Diğer frame'ler için, IOU tabanlı takip yap
        elif len(current_boxes) > 0 and len(prev_boxes) > 0:
            current_ids = [-1] * len(current_boxes)

            # Her mevcut kutu için en iyi eşleşmeyi bul
            for i, current_box in enumerate(current_boxes):
                best_iou = 0.3  # IOU eşiği
                best_id = -1

                for j, prev_box in enumerate(prev_boxes):
                    iou = calculate_iou(current_box, prev_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_id = prev_ids[j]

                if best_id != -1:
                    current_ids[i] = best_id
                else:
                    # Eşleşme bulunamazsa yeni ID ata
                    current_ids[i] = next_id
                    next_id += 1
        else:
            current_ids = []

        # Tespit edilen oyuncuları kaydet
        for i, box in enumerate(current_boxes):
            player_id = current_ids[i]
            x1, y1, x2, y2 = map(int, box)

            # Yeni bir oyuncu ise, sözlüğe ekle
            if player_id not in players:
                players[player_id] = []

            # Oyuncunun bu frame'deki görüntüsünü kaydet
            player_crop = frame[y1:y2, x1:x2]
            frame_info = {
                'frame_num': frame_idx,
                'crop': player_crop,
                'box': (x1, y1, x2, y2)
            }
            players[player_id].append(frame_info)

        # Mevcut frame'i bir sonraki işlem için sakla
        prev_boxes = current_boxes
        prev_ids = current_ids

        frame_idx += 1

    cap.release()

    # Çok az frame'de görünen oyuncuları (muhtemelen yanlış tespitler) filtrele
    filtered_players = {k: v for k, v in players.items() if len(v) > 5}

    print(f"Sahne {scene_id} için {len(filtered_players)} oyuncu tespit edildi ve takip edildi.")
    return filtered_players

def calculate_iou(box1, box2):
    """
    İki bounding box arasındaki IOU (Intersection over Union) değerini hesaplar
    """
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Kesişim alanını hesapla
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)

    if x2_i < x1_i or y2_i < y1_i:
        return 0.0

    intersection_area = (x2_i - x1_i) * (y2_i - y1_i)

    # Birleşim alanını hesapla
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)

    union_area = box1_area + box2_area - intersection_area

    return intersection_area / union_area

def extract_jersey_region(player_frames):
    """
    Oyuncunun her frame'i için forma bölgesini tespit eder
    Args:
        player_frames: Oyuncunun frame bilgilerini içeren liste
    Returns:
        jersey_crops: Forma bölgesi kesitlerini içeren liste
    """
    jersey_crops = []

    for frame_info in player_frames:
        crop = frame_info['crop']

        if crop.size == 0 or crop.shape[0] < 10 or crop.shape[1] < 10:
            continue

        # Forma bölgesi, genellikle vücudun üst kısmındadır
        h, w = crop.shape[:2]

        # Üst 1/3 ve orta 1/3 bölgeyi al (forma genellikle burada olur)
        jersey_region = crop[int(h*0.15):int(h*0.6), int(w*0.1):int(w*0.9)]

        if jersey_region.size == 0 or jersey_region.shape[0] < 10 or jersey_region.shape[1] < 10:
            continue

        # Görüntü iyileştirme
        jersey_region = enhance_image(jersey_region)

        jersey_crops.append(jersey_region)

    return jersey_crops

def enhance_image(image):
    """
    Forma numarasının daha iyi algılanması için görüntüyü iyileştirir
    """
    # Görüntü boyutu çok küçükse yeniden boyutlandır
    if image.shape[0] < 60 or image.shape[1] < 60:
        scale_factor = max(60 / image.shape[0], 60 / image.shape[1])
        new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
        image = cv2.resize(image, new_size, interpolation=cv2.INTER_CUBIC)

    # Gri tonlamaya dönüştür
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Adaptif eşikleme uygula
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY_INV, 11, 2)

    # Morfolojik işlemlerle gürültüyü azalt
    kernel = np.ones((2, 2), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Kenarları belirginleştir
    edges = cv2.Canny(gray, 50, 150)

    # Orijinal görüntüyü kontrastını artır
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_gray = clahe.apply(gray)

    # BGR'a dönüştür
    enhanced = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)

    return enhanced

def recognize_jersey_number(jersey_crops):
    """
    Forma kesitlerinden numara tespiti yapar
    Args:
        jersey_crops: Forma bölgesi kesitlerinin listesi
    Returns:
        jersey_number: En çok tespit edilen forma numarası
    """
    if not jersey_crops:
        return None

    numbers = []

    for jersey in jersey_crops:
        # Tesseract ile OCR uygula
        # Sadece rakamları tespit etmek için konfigürasyon ayarı
        text = pytesseract.image_to_string(jersey, config='--psm 6 -c tessedit_char_whitelist=0123456789')

        # Bulunan metni temizle ve sayıları çıkar
        if text.strip():
            try:
                # İlk bulunan sayıyı al
                for char in text:
                    if char.isdigit():
                        numbers.append(int(char))
                        break
            except:
                pass

    # En sık tespit edilen numarayı bul (majority voting)
    if numbers:
        counter = Counter(numbers)
        most_common = counter.most_common(1)
        return most_common[0][0]

    return None

def process_video(video_path, output_dir='output'):
    """
    Videoyu tam olarak işleyen ana fonksiyon
    Args:
        video_path: İşlenecek video dosyasının yolu
        output_dir: Sonuçların kaydedileceği dizin
    """
    # Çıktı dizinini oluştur
    os.makedirs(output_dir, exist_ok=True)

    # Modelleri indir
    download_models()

    # Sahneleri tespit et
    scenes = detect_scenes(video_path)

    # Her sahne için oyuncuları tespit et ve takip et
    all_players = {}

    for i, scene in enumerate(scenes):
        print(f"\nSahne {i+1}/{len(scenes)} işleniyor...")
        players = detect_and_track_players(video_path, scene, i+1)

        # Her oyuncu için forma bölgesini çıkar ve numarayı tespit et
        scene_players = {}

        for player_id, frames in players.items():
            jersey_crops = extract_jersey_region(frames)

            if jersey_crops:
                jersey_number = recognize_jersey_number(jersey_crops)

                if jersey_number is not None:
                    scene_players[player_id] = {
                        'jersey_number': jersey_number,
                        'frames': frames
                    }

        all_players[f"scene_{i+1}"] = scene_players

    # Sonuçları göster ve kaydet
    results = {}

    for scene_id, players in all_players.items():
        scene_results = {}

        for player_id, player_info in players.items():
            jersey_number = player_info['jersey_number']
            scene_results[f"player_{player_id}"] = jersey_number

        results[scene_id] = scene_results

    # Sonuçları ekrana yazdır
    print("\n--- Forma Numarası Tanıma Sonuçları ---")
    for scene_id, players in results.items():
        print(f"\n{scene_id}:")
        for player_id, jersey_number in players.items():
            print(f"  {player_id}: {jersey_number}")

    # Sonuçları görselleştir ve kaydet
    visualize_results(video_path, all_players, output_dir)

    return results

def visualize_results(video_path, all_players, output_dir):
    """
    Tespit sonuçlarını görselleştirir ve kaydeder
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Sonuç videosu için
    result_path = os.path.join(output_dir, "result_video.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(result_path, fourcc, fps, (width, height))

    # Her sahneyi işle
    for scene_id, players in all_players.items():
        # Oyuncuların görüldüğü tüm frame'leri topla
        frame_to_players = {}

        for player_id, player_info in players.items():
            jersey_number = player_info['jersey_number']

            for frame_info in player_info['frames']:
                frame_num = frame_info['frame_num']
                box = frame_info['box']

                if frame_num not in frame_to_players:
                    frame_to_players[frame_num] = []

                frame_to_players[frame_num].append((player_id, jersey_number, box))

        # Tüm frame numaralarını sırala
        frame_nums = sorted(frame_to_players.keys())

        if not frame_nums:
            continue

        # Her frame'i işle
        for frame_num in frame_nums:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()

            if not ret:
                continue

            # Frame'deki tüm oyuncuları çiz
            for player_id, jersey_number, box in frame_to_players[frame_num]:
                x1, y1, x2, y2 = box

                # Bounding box çiz
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Forma numarasını yazdır
                label = f"#{jersey_number}"
                cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Sahne ID'sini yazdır
            cv2.putText(frame, scene_id, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Frame'i kaydet
            out.write(frame)

    cap.release()
    out.release()

    print(f"Sonuç videosu kaydedildi: {result_path}")

# Ana çalıştırma bloğu
if __name__ == "__main__":
    # Google Drive'a bağlan (isteğe bağlı)
    from google.colab import drive
    drive.mount('/content/drive')

    # Video yolunu belirt
    video_path = "/content/drive/MyDrive/videos/demo.mp4"  # Bunu kendi video yolunuzla değiştirin

    # Videoyu işle
    results = process_video(video_path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 30.000 FPS, resolution: 852 x 480
INFO:pyscenedetect:Detecting scenes...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
YOLO modellerini indirme...
Sahneleri tespit etme...
Toplam 6 sahne tespit edildi.

Sahne 1/6 işleniyor...
Sahne 1 için oyuncuları tespit etme ve takip etme...

0: 384x640 18 persons, 2929.5ms
Speed: 4.2ms preprocess, 2929.5ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 2841.5ms
Speed: 4.3ms preprocess, 2841.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 2863.0ms
Speed: 4.0ms preprocess, 2863.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 4378.9ms
Speed: 4.4ms preprocess, 4378.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 3158.9ms
Speed: 4.2ms preprocess, 3158.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 2857.9ms
Speed: 4.0ms prepr

IndexError: The shape of the mask [154] at index 0 does not match the shape of the indexed tensor [153, 6] at index 0